In [2]:
import numpy as np
import pickle
import pandas as pd
import sys
sys.path.append('..')
sys.path.append('../analyses')

import analysis
import datatypes
import qm9_filter_generated

ModuleNotFoundError: No module named 'pybel'

In [ ]:
methane = analysis.construct_molecule('CH3')[0]
valence = np.array([0, 1, 0, 0, 0, 0, 4, 3, 2, 1])

In [5]:
qm9_filter_generated.check_valency(np.asarray([methane.positions]), np.asarray([list(methane.numbers)]), valence)

{'mols': [<utility_classes.Molecule at 0x107957850>],
 'connectivity': array([[[0., 1., 1., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]]]),
 'valid_mol': array([False]),
 'valid_atom': array([0.75])}

In [3]:
config, best_state, _, metrics_for_best_state = analysis.load_from_workdir('../workdirs/v5/mace/interactions=4/l=5/channels=32')

In [4]:
metrics_for_best_state

{'train_eval_final': {'total_loss': array(-1.0350323, dtype=float32),
  'focus_loss': array(0.25593817, dtype=float32),
  'atom_type_loss': array(0.38758564, dtype=float32),
  'position_loss': array(-1.6785557, dtype=float32)},
 'val_eval_final': {'total_loss': array(-0.58744246, dtype=float32),
  'focus_loss': array(0.24660453, dtype=float32),
  'atom_type_loss': array(0.3918947, dtype=float32),
  'position_loss': array(-1.2259465, dtype=float32)},
 'test_eval_final': {'total_loss': array(-0.68538874, dtype=float32),
  'focus_loss': array(0.30432183, dtype=float32),
  'atom_type_loss': array(0.535974, dtype=float32),
  'position_loss': array(-1.5256858, dtype=float32)}}

In [11]:
df_dict = analysis.get_results_as_dataframe(
    ['mace'], ['focus_loss', 'atom_type_loss', 'position_loss'], '../workdirs'
)

In [13]:
df_dict['train_eval_final']

,config.activation,config.avg_num_neighbors,config.compute_padding_dynamically,config.eval_every_steps,config.focus_predictor.latent_size,config.focus_predictor.num_layers,config.learning_rate,config.learning_rate_schedule,config.learning_rate_schedule_kwargs.decay_steps,config.learning_rate_schedule_kwargs.init_value,...,config.val_molecules,focus_loss,atom_type_loss,position_loss,model,max_l,num_interactions,num_channels,num_params,num_train_molecules
0,softplus,15.0,False,5000,128,3,0.001,sgdr,1000,0.001,...,47616,0.255938,0.387586,-1.678556,mace,5,4,32,1227040,47616
